In [1]:
import tensorflow as tf
import os

In [7]:
os.path.join('hairdata', 'bald')
os.listdir('hairdata')

['notbald', '.DS_Store', 'bald']

In [8]:
# Remove dodgy images
import cv2
import imghdr
from matplotlib import pyplot as plt

In [9]:
data_dir = 'hairdata'

In [10]:
image_exts = ['jpeg','jpg','bmp','png']

In [12]:
img = cv2.imread(os.path.join('hairdata','bald','423138845.jpg'))
img.shape

[ WARN:0@164.764] global /Users/runner/miniforge3/conda-bld/libopencv_1648506022632/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('hairdata/bald/423138845.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
# Load data
tf.data.Dataset??

In [ ]:
import numpy as np

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('hairdata')

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
data_iterator

In [ ]:
# Get another batch from the iterator
batch = data_iterator.next()

In [ ]:
# Images represented as numpy arrays
batch[0].shape

In [ ]:
# Class 1 = Sad People
# Class 0 = Happy People 
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# Preprocess Data
# Scale data
data = data.map(lambda x, y: (x/225, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()


In [ ]:
scaled_iterator

In [ ]:
batch = scaled_iterator.next()

In [ ]:
batch[0].min()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# Split data
len(data)

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)+1

In [ ]:
train_size + val_size + test_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)


In [ ]:
# Build Deep learning model
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout


In [ ]:

model = Sequential()

In [ ]:

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])


In [ ]:

model.summary()

In [ ]:
# Train

logdir='logs'

In [ ]:

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val)

In [ ]:
# Plot Performance

fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Evaluate
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy


In [ ]:

pre = Precision()
re = Recall()
acc = BinaryAccuracy()


In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:

print(pre.result(), re.result(), acc.result())


In [ ]:
#Test
import cv2

In [ ]:

img = cv2.imread('sithudo.jpeg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))


In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is Happy')

In [ ]:
# Save the model

from tensorflow.keras.models import load_model


In [ ]:
model.save(os.path.join('models','hairimageclassifier.h5'))

In [ ]:
new_model = load_model('models/hairimageclassifier.h5')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))
